In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

import torch
#from bertopic import BERTopic
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer

from datetime import datetime
from tqdm.auto import tqdm
import html

sns.set(palette='summer')

## Load data

In [6]:
data = pd.read_json('../data/geo-reviews-dataset-2023.jsonl', lines=True)

In [7]:
data.head(2)

,address,name_ru,rating,rubrics,text,#_words,#_chars
0,"Республика Мордовия, Ковылкино, улица Ленина, 2А",СберБанк,5,Банк,"Всё отлично, вежливый персонал, талонная систе...",34,235
1,"Санкт-Петербург, улица Коллонтай, 12, корп. 1",Пятёрочка,5,Супермаркет,"Чистота, без просроченных продуктов, персонал ...",8,80


In [8]:
len(data)

19137

In [30]:
data["name_ru"].value_counts()

Пятёрочка           5946
Магнит              2575
Красное&Белое       1690
Wildberries         1656
Ozon                1469
Вкусно — и точка    1165
Перекрёсток         1141
Fix Price           1089
Пляж                1082
СберБанк            1009
Name: name_ru, dtype: int64

In [31]:
data = data[data["name_ru"] == "Магнит"]

## Simple preprocessing

In [32]:
data = data[data['#_words'] > 4]
data['text'] = data['text'].apply(
    lambda x: " ".join(x.lower().replace('ё', 'е').strip().split())
)

data['text'] = data['text'].apply(html.unescape)
len(data)

2575

In [33]:
data['text'].nunique()

2575

## Sentiment analysis

https://huggingface.co/Tatyana/rubert-base-cased-sentiment-new

In [34]:
model = AutoModelForSequenceClassification.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')
tokenizer = AutoTokenizer.from_pretrained('Tatyana/rubert-base-cased-sentiment-new')

In [35]:
scores = []
batch_size = 128
device = 'cuda:5'

model = model.to(device)
model.eval()

with torch.no_grad():
    for i in tqdm(range(0, len(data), batch_size)):
        batch = tokenizer(
            data['text'].iloc[i:i+batch_size].tolist(), 
            truncation=True, padding='longest', return_tensors='pt', max_length=512)
        batch = {k: v.to(device) for k, v in batch.items()}
        
        preds = model(**batch).logits.argmax(dim=1)
        scores.append(preds)

  0%|          | 0/21 [00:00<?, ?it/s]

In [36]:
preds_mapping = {
    0: "NEUTRAL",
    1: "POSITIVE",
    2: "NEGATIVE",
}

In [37]:
data['sentiment'] = torch.cat(scores).tolist()
data['sentiment'] = data['sentiment'].apply(lambda x: preds_mapping[x])

In [38]:
data['sentiment'].value_counts()

POSITIVE    1944
NEGATIVE     403
NEUTRAL      228
Name: sentiment, dtype: int64

## Clusterization

In [47]:
def get_clusters(sample_data: pd.DataFrame) -> pd.DataFrame:
    bert = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    bert.eval()
    topic_model = BERTopic(embedding_model=bert, verbose=True, min_topic_size=10, calculate_probabilities=True)
    
    embeddings = bert.encode(sample_data['text'].tolist(), show_progress_bar=True)
    topics, probs = topic_model.fit_transform(sample_data['text'].tolist(), embeddings=embeddings)
    
    sample_data['topics'] = topics
    #sample_data = sample_data.drop(columns=['simple_text', 'platform'])
    
    topic_info = topic_model.get_topic_info()[['Topic', 'Name']]
    topic_info.columns = ['topics', 'keywords']
    topic_info['keywords'] = topic_info['keywords'].apply(lambda x: ", ".join(x.split('_')[1:]))
    
    sample_data = sample_data.merge(topic_info, how='left', on='topics')
    
    return sample_data

In [48]:
data = data.drop_duplicates(subset=["text"])

In [49]:
clusters = get_clusters(data)

Batches:   0%|          | 0/81 [00:00<?, ?it/s]

2023-12-05 20:06:23,349 - BERTopic - Reduced dimensionality
2023-12-05 20:06:23,554 - BERTopic - Clustered reduced embeddings


In [50]:
clusters["topics"].value_counts()

-1     907
 0     490
 1     178
 2     136
 3     132
 4     109
 5      89
 6      86
 7      62
 8      47
 9      43
 10     42
 11     42
 12     37
 13     35
 14     26
 15     22
 16     20
 17     18
 18     17
 19     14
 20     13
 21     10
Name: topics, dtype: int64

In [60]:
clusters.to_json("../data/magnit-clusters.jsonl", lines=True, orient="records", force_ascii=False)

## Cluster analysis

In [55]:
#topic_model.get_topic_info()

In [56]:
#topic_model.get_document_info(sample['text'])

In [58]:
#topic_model.get_representative_docs()

In [186]:
#topic_model.merge_topics(docs, topics_to_merge)

## Cluster visualization

In [59]:
#topic_model.visualize_topics()

In [60]:
#topic_model.visualize_documents(sample['text'].tolist())

In [216]:
#hierarchical_topics = topic_model.hierarchical_topics(sample['text'].tolist())

#topic_model.visualize_hierarchical_documents(sample['text'].tolist(), hierarchical_topics)

In [61]:
#topic_model.visualize_hierarchy()

In [62]:
#topic_model.visualize_barchart(topics=[0, 5])

In [63]:
#topic_model.visualize_heatmap()

In [197]:
#topic_model.visualize_term_rank()

#### Topics with lowest sentiment

In [38]:
# bad_topics = ranking.sort_values(by='numeric_sentiment').iloc[:5].index

# for topic in bad_topics:
#     print('=' * 40)
#     for text in sample[sample['topics'] == topic]['text']:
#         print(text)

#### Topics with highest sentiment

In [39]:
# bad_topics = ranking.sort_values(by='numeric_sentiment', ascending=False).iloc[:5].index

# for topic in bad_topics:
#     print('=' * 40)
#     for text in sample[sample['topics'] == topic]['text']:
#         print(text)

#### Topics with longest answers

In [40]:
# bad_topics = ranking.sort_values(by='num_words', ascending=False).iloc[:5].index

# for topic in bad_topics:
#     print('=' * 40, topic)
#     for text in sample[sample['topics'] == topic]['text']:
#         print(text)

### Summarization with ChatGPT

Получение осмысленных коротких описаний для отдельных кластеров

In [2]:
import openai

In [3]:
openai.api_key = # api key

In [4]:
clusters = pd.read_json("../data/magnit-clusters.jsonl", lines=True)

In [6]:
#clusters

In [75]:
def get_topic_summary(df, topic_id, total_samples=7, model="gpt-3.5-turbo") -> str:
    keywords = df[df['topics'] == topic_id]["keywords"].iloc[0]
    prompt = "Какие выводы можно сделать по этиму кластеру отзывов пользователей?\n\n"
    prompt += f"Ключевые слова для кластера: {keywords}\n\n"

    for i, x in enumerate(df[df['topics'] == topic_id]['text'].sample(total_samples)):
        prompt += f"{i+1}. {x.strip()}.\n"

    prompt += "В ответ напиши один ключевой вывод."
    #print(prompt)
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    
    answer = response.choices[0].message.content
    
    return answer

In [76]:
#clusters["keywords"]

In [77]:
report = []
sentiment_values = ["POSITIVE", "NEGATIVE", "NEUTRAL"]


for topic_id in tqdm(sorted(clusters["topics"].unique())):
    topic_representation = get_topic_summary(clusters, topic_id)
    
    sub_df = clusters[clusters["topics"] == topic_id]
    
    current_line = [topic_representation, len(sub_df)]
    current_line.extend(
        [(sub_df["sentiment"] == sent).sum() / len(sub_df) for sent in sentiment_values]
    )
    
    report.append(current_line)

  0%|          | 0/23 [00:00<?, ?it/s]

In [78]:
#topic_representation = get_topic_summary(clusters, 1)

In [79]:
#topic_representation

In [80]:
report = pd.DataFrame(report)

In [81]:
report.columns = ["Topic", "Total"] + sentiment_values

In [82]:
report["Topic"] = report["Topic"].apply(lambda x: x.lstrip("Тема: ").strip())

In [83]:
for col in sentiment_values:
    report[col] = report[col].apply(lambda x: round(x, 2))

In [85]:
for topic in report["Topic"]:
    print(topic)
    print('-' * 10)

Один из ключевых выводов можно сделать, что большинство пользователей оценивают магазин положительно, отмечая чистоту, доброжелательность персонала и хорошее размещение товаров. Однако, некоторые отзывы указывают на неудовлетворенность некачественными товарами, несоответствующими ценниками, грубостью кассиров и неудобным расположением магазина.
----------
Судя по данному кластеру отзывов пользователей, одним из ключевых выводов можно сделать то, что магнит имеет проблемы с качеством обслуживания и точностью расчетов, что вызывает недовольство у некоторых клиентов.
----------
Один ключевой вывод по данному кластеру отзывов пользователей может быть следующим: большинство пользователей положительно относится к доступности парковки, комфортной покупке в магазине и его удобному расположению.
----------
Кластер отзывов пользователей говорит о положительном отношении качества овощей и фруктов в магазине, а также обращает внимание на профессионализм и отзывчивость персонала.
----------
Магазин

In [28]:
report.to_csv("../data/magnit_report.csv", index=False)

In [30]:
report

,Topic,Total,POSITIVE,NEGATIVE,NEUTRAL
0,Отзывы о магазине и его персонале,907,0.77,0.16,0.08
1,"Отзывы о магазине ""Магнит""",490,0.76,0.12,0.11
2,Положительные отзывы о магазине и его преимуще...,178,0.93,0.02,0.05
3,Отзывы о магазине и его продуктах.,136,0.74,0.15,0.11
4,Отзывы о магазине средней ценовой политики с п...,132,0.85,0.05,0.10
5,Негативный опыт посещения магазина,109,0.34,0.52,0.14
6,Негативный опыт обслуживания в магазине,89,0.15,0.80,0.06
7,Положительный опыт покупок в магазине,86,0.91,0.01,0.08
8,Положительные отзывы о магазине и персонале,62,0.94,0.03,0.03
9,Приятный магазин с хорошим обслуживанием и асс...,47,1.00,0.00,0.00


In [34]:
clusters.sort_values(by="topics").to_csv("../data/magnit-clusters.csv", index=False)